<a href="https://colab.research.google.com/github/francji1/01PR/blob/main/R/01PR_Ex03_sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Settings and requirements:

In [ ]:
######################
# get requirements for Lecture 1
list_of_packages <- c("tidyverse", "ascentTraining","GGally","knitr","pander")
missing_packages <- list_of_packages[!(list_of_packages %in% installed.packages()[,"Package"])]
if(length(missing_packages)) install.packages(missing_packages)
lapply(list_of_packages, library, character.only = TRUE)

######################
# check our settings
#getwd()
#print(.libPaths())
print(sessionInfo())
print(version)

In [ ]:
knitr::opts_chunk$set(echo = FALSE)

## Popis úlohy

V tomto úkolu je cílem provést předzpracování datového souboru, jeho vizualizaci a jednoduchou lineární regresní úlohu, kde budeme modelovat spotřebu automobilu v závislosti na jeho váze. K tomuto účelu poslouží datový souboru *auto\_mpg* z balíčku ascentTraining  který obsahuje 398 pozorování o 10 proměnných. Dataset byl prvně použit americkou statistickou společností v roce  1983 a lze ho též najít na  UCI Machine Learning Repository, případně na kaggle.com s několika pracovními sešity.


In [ ]:
? auto_mpg 
auta <- auto_mpg

In [ ]:
knitr::kable(head(auto_mpg))
#pander(head(auto_mpg))

In [ ]:
auto_mpg %>% head()

## Úprava dat

## Otázka 01

Zjistěte, zdali data neobsahují chybějící hodnoty (*NA*). Pokud ano, tak rozhodněte zdali můžete příslušná pozorování z dat odstranit a proč. Které proměnné jsou kvantitativní a které kvalitativní? Jeli možno některé zařadit do obou skupin, pro kterou se rozhodnete? Které proměnné budete brát jako faktorové a proč? Spočtěte základní statistiky pro jednotlivé proměnné.


In [ ]:
colSums(is.na(auta))

In [ ]:
auta[is.na(auta$horsepower),]

In [ ]:
str(auta)

In [ ]:
table(auta$cylinders)

In [ ]:
table(auta$model_year)

In [ ]:
auta %>% summary()

In [ ]:
auta2 <- auta %>%
            mutate(origin = as.factor(origin),
            make = as.factor(make),
            car_name =as.factor(car_name))
 
auta2 %>%  summary()

## Otázka 02

Proměnnou *mpg* nahraďte proměnnou *spotreba* kde bude místo počtu ujetých mil na galon paliva uvedena hodnota počet litrů na 100 Km.  Proměnnou *cylinders* přejmenujte na  *pocet\_valcu*. Proměnnou *displacement* přejmenujte na  *zdvihovy\_objem* a převeďte z kubických palců na litry. Proměnnou  *horsepower* přejmenujte na *výkon* a převeďte na kW. Proměnnou *weight* přejmenujte na  *hmotnost* a převeďte z liber na kilogramy. Odstraňte proměnnou *acceleration*. Proměnnou *model.year* přejmenujte na *rok\_vyroby* a upravte ji tak, aby její hodnoty popisovaly celý rok 19XX. Proměnnou *origin* přejmenujte na *puvod* a upravte ji tak, že místo North America bude USA, místo Europe EUR a místo Asia JAP. Z proměnné  *car.name* vytvořte proměnnou  *vyrobce* podle prvního slova obsaženého v řetězci proměnné  *car.name*.

In [ ]:
auta$spotreba = (auta$mpg)

In [ ]:
head(auta)

In [ ]:
levels(factor(auta$origin))

In [ ]:
auta3 <- auta %>%
  rename(spotreba = mpg,
        pocet_valcu = cylinders,
        zdvihovy_objem = displacement,
        vykon = horsepower,
        hmotnost = weight,
        rok_vyroby = model_year,
        puvod = origin,
        vyrobce = make,
        model = car_name) %>%
  mutate(spotreba =  100/(1.609344*(spotreba/3.78541178)),
        zdvihovy_objem = zdvihovy_objem/(2.54^3),
        vykon = vykon*0.74569987,
        hmotnost = hmotnost*0.45359237,
        rok_vyroby = 1900+rok_vyroby,
        vyrobce_check = as.factor(word(model,1)),
        model=as.factor(model),
        vyrobce=as.factor(vyrobce),
        puvod = factor(puvod, labels= c("JPN","EUR","USA")) # vim poradi jednotlivych levels
        ) 

#levels(auta3$puvod) <- c("JPN","EUR","USA")
#auta3$puvod[auta3$puvod == "Asia"] = "JPN"
#auta3$puvod[auta3$puvod == "Europe"] = "EUR"
#auta3$puvod[auta3$puvod == "North America"] = "USA"

  auta3 %>% summary()

In [ ]:
head(auta3)

In [ ]:
? factor

In [ ]:
auta3[as.character(auta3$vyrobce) != as.character(auta3$vyrobce_check), ]

In [ ]:
1/(2.54^3)

In [ ]:
? word

# Vizualizace dat

## Otázka 03 

Vykreslete scatterploty pro všechny numerické proměnné. Pro proměnné *spotreba* a *hmotnost* vykreslete histogramy spolu s jádrovými odhady hustot. Pro proměnné *pocet\_valcu* a *rok\_vyroby* vykreslete krabicové diagramy, kde odezvou bude *spotreba*.  Je z těchto grafů vidět, že některá auta mají jinou, než očekávanou spotřebu? Navrhněte úpravu těchto dvou proměných (případně úpravu datasetu) tak, aby obě proměnné  *pocet\_valcu* a *rok\_vyroby* byly faktorové a obsahovaly právě 3 úrovně. Pro takto upravená data vykreslete místo výše zmííněných boxplotů violin ploty.

In [ ]:
auta4 <- auta3[,1:8]


In [ ]:
ggpairs(auta4)

In [ ]:
ggplot(auta3,aes(x=factor(pocet_valcu),y=spotreba, fill=puvod)) +
  geom_boxplot() +
  #geom_jitter() +
  xlab("Pocet valcu") +
  ylab("Spotreba")

In [ ]:
ggplot(auta3,aes(x=factor(rok_vyroby),y=spotreba, fill=puvod)) +
  geom_boxplot() +
  #geom_jitter() +
  xlab("Rok vyroby") +
  ylab("Spotreba")

In [ ]:
auta4 <- auta3 %>%
  mutate(rok_vyroby = factor(cut(rok_vyroby, c(1969,1975,1979,1985)), label=c("70-75","75-79","80-85"))
        )
auta4 %>% summary()        

In [ ]:
ggplot(auta4,aes(x=rok_vyroby,y=spotreba, fill=puvod)) +
  geom_violin() +
  geom_jitter(aes(colour = puvod)) +
  xlab("Rok vyroby") +
  ylab("Spotreba")

In [ ]:
? geom_jitter

## Otázka 04

Pro kombinace faktorizovaných proměnných *pocet\_valcu*, *rok\_vyroby* a *puvod* vykreslete spotřebu aut, aby bylo na obrázku vidět, jestli se liší spotřeba u aut pocházejících z různých kontinentů v závislosti na počtu válců, roku výroby a naopak. Zobrazte jen kombinace s relevantním počtem dat.

In [ ]:
auta4

## Otázka 05

Pro auta výrobce Chrysler vykreslete závislost spotřeby na váze automobilu, kde jednotlivé události označíte barvou podle počtu válců a velikost bodů v grafu bude odpovídat objemu motoru.

In [ ]:
#auta_Ch <- auta4 %>% filter(vyrobce == "chrysler")
table(auta4$vyrobce)
ggplot(auta4 %>% filter(vyrobce == "chevrolet"), aes(x=hmotnost, y = spotreba, col = factor(pocet_valcu), size = zdvihovy_objem) ) +
  geom_point()